We use this script to process the Gesture dataset and get the input files for the GRU-D models.

In [1]:
import os 
import numpy as np
import pandas as pd
from python_linq import From

In [2]:
seed = 42
np.random.seed(seed=seed)

In [3]:
gesture_root_folder = os.path.join('gesture_datas/')
grud_working_folder = '.' # # This should be the [WD] (working directory) of the GRU-D project.
grud_gesture_data_folder = os.path.join(grud_working_folder, 'data', 'gesture')
if not os.path.exists(grud_gesture_data_folder):
    os.makedirs(grud_gesture_data_folder)
print('The data files will be converted and saved here: ', grud_gesture_data_folder)

The data files will be converted and saved here:  .\data\gesture


In [4]:
def Load_data(path):
    all_files = os.listdir(path)
    files_to_load = From(all_files).where(lambda file: file[-3:]=='csv').where(lambda file: file[-5]=='w').toList()
    df_Big = pd.read_csv(path +files_to_load[0], sep=',')
    for file in files_to_load:
        df_file = pd.read_csv(path + file, sep=',')
        df_Big = df_Big.append(df_file)
    df_Big = df_Big.loc[df_Big['phase']!='Preparação']
    return df_Big
dataframe = Load_data(gesture_root_folder)

In [5]:
def Truncate(df):
    k = 0
    nb_rows = df.shape[0]
    df_to_fill = pd.DataFrame(columns=df.columns.values.tolist())
    while k < nb_rows:    
        row_considered = df.iloc[[k]]
        phase_considered = row_considered['phase'].values.tolist()[0]
        phase_tmp = phase_considered
        #print(phase_considered)
        #print(phase_tmp)
        i = 0
        while phase_tmp == phase_considered and i<30:
            if k + i + 1 < nb_rows:
                # Verifying if there is a next value
                i += 1
                phase_tmp = df.iloc[[i+k]]['phase'].values.tolist()[0]
            else:
                # End of while
                break
        if i > 7:
            # Add the row considered            
            df_to_fill = df_to_fill.append(row_considered)            
        if k + i + 1 < nb_rows:
            k += i
        else:
            # End of the dataframe
            break
    df_to_fill = df_to_fill.reset_index(drop=True)
    return df_to_fill
df_truncated = Truncate(dataframe)

In [6]:
def Creat_missing_values(df, nb_rows_to_del=1):
    nb_rows = df.shape[0]
    nb_columns = df.shape[1]
    index_still = np.arange(nb_rows)
    df_copy = df.copy()
    if nb_rows_to_del < nb_rows:
        while nb_rows-len(index_still)<nb_rows_to_del:
            index_del = np.random.randint(len(index_still)-1)
            timestamp_saved = df.iloc[index_del].__getitem__('timestamp')
            df_copy.iloc[index_del] = [np.nan for i in range(nb_columns)]
            df_copy.loc[index_del, 'timestamp'] = timestamp_saved
            index_still = np.delete(index_still,index_del)
    return df_copy
df_truncated_missing_data = Creat_missing_values(df_truncated)

In [7]:
def Creat_mask(df):
    mask = df_truncated_missing_data.copy()
    mask = mask.where(mask.isnull(),1)
    mask = mask.where(mask.isnull()!=True,0)
    return mask
Mask = Creat_mask(df_truncated_missing_data)

In [16]:
df_truncated_missing_data.loc[:,'lhx':'rwz'].values


array([[5.347435, 4.363681, 1.501913, ..., 5.553945, 4.370456, 1.553521],
       [5.17276 , 4.29817 , 1.539378, ..., 5.101995, 4.275612, 1.545337],
       [4.900373, 4.314654, 1.532006, ..., 5.195107, 4.343143, 1.542306],
       ...,
       [3.602747, 5.023941, 1.837341, ..., 5.411085, 1.416187, 1.841328],
       [3.611109, 5.022792, 1.832637, ..., 5.370014, 1.446505, 1.81172 ],
       [3.896576, 5.386982, 1.763772, ..., 5.238472, 4.971535, 1.822406]])

In [17]:
timestamp = df_truncated_missing_data.timestamp
processed_data = {
    'input': df_truncated_missing_data.loc[:,'lhx':'rwz'].values,
    'masking': Mask
}
n = len(timestamp)

In [18]:
# process label
phase = ['Rest', 'Preparation', 'Hold', 'Stroke', 'Retraction']
processed_label = {
    'label_phase': phase
}

In [19]:
np.savez_compressed(os.path.join(grud_gesture_data_folder, 'data.npz'), timestamp=timestamp, **processed_data, **processed_label)

In [31]:
from sklearn.model_selection import train_test_split, KFold, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
#https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-of-time-series-data

In [32]:
kf = TimeSeriesSplit(n_splits=2)
for train, test in kf.split(processed_data['input']):
     print("train:%s \ntest:%s" % (train, test))

train:[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179] 
test:[180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 2

In [ ]:
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.25)

In [ ]:
#processed_fold = {
#    'fold_phase': fold_data['folds_ep_pha'][0][0]    
#}

In [ ]:
#np.savez_compressed(os.path.join(grud_gesture_data_folder, 'fold.npz'), **processed_fold)